In [2]:
pip install torch --ignore-installed TBB

  Using cached torch-2.3.1-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Using cached tbb-2021.12.0-py3-none-win_amd64.whl.metadata (1.1 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
Using cached torch-2.3.1-cp311-cp311-win_amd64.whl (159.8 MB)
Using cached tbb-2021.12.0-py3-none-win_amd64.whl (286 kB)
Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl (3.5 MB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
   ---------------------------------------- 0.0/176.9 kB ? eta -:--:--
   --------- ----------------------------- 41.0/176.9 kB 991.0 kB/s eta 0:00:01
   -------------------- ------------------- 92.2/176.9 kB 1.1 MB/s eta 0:00:01
   ---------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2024.6.0 which is incompatible.


In [1]:
import streamlit as st
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2ForSequenceClassification
import torch
import librosa
import numpy as np
import noisereduce as nr
import os

c:\Users\lyes_\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
# Cargar modelos y procesadores
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weig

# Funciones de preprocesamiento de audio

In [8]:
def preprocess_audio(file_path, target_sr=16000):
    audio, sr = librosa.load(file_path, sr=target_sr, mono=True)
    return audio, sr

In [5]:
def normalize_audio(audio):
    max_amp = np.max(np.abs(audio))
    normalized_audio = audio / max_amp
    return normalized_audio

In [6]:
def denoise_audio(audio, sr):
    reduced_noise = nr.reduce_noise(y=audio, sr=sr)
    return reduced_noise

In [7]:
def segment_audio(audio, sr, segment_length=5):
    segments = []
    total_duration = librosa.get_duration(audio, sr)
    for start in range(0, int(total_duration), segment_length):
        end = start + segment_length
        segment = audio[start*sr:end*sr]
        segments.append(segment)
    return segments

# Función de transcripción de audio

In [9]:
def transcribe_audio(audio, sr):
    input_values = processor(audio, sampling_rate=sr, return_tensors="pt").input_values
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription

# Función de análisis de tono (placeholder)

In [10]:
def analyze_tone(transcription):
    # Placeholder para el análisis de tono basado en la transcripción
    if "angry" in transcription.lower():
        return "angry"
    elif "happy" in transcription.lower():
        return "happy"
    else:
        return "neutral"

# Función de análisis de texto

In [11]:
def analyze_text(text, tone):
    # Placeholder para el análisis de texto
    analysis = f"Análisis del texto con tono {tone}"
    return analysis